
# 1.Importe de Archivos SECOP I

In [ ]:
# Upgrade packaeges in python
import sys
!pip install sodapy
!{sys.executable} -m pip install -U pandas-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension
!pip uninstall pandas_profiling
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

!pip install plotly>=4.0.0
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4


In [ ]:
#Import libraries
!pip install sodapy

from sodapy import Socrata
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import datetime
from pathlib import Path
from ipywidgets import widgets
from pandas_profiling import ProfileReport
from datetime import datetime

import re
from itertools import tee, islice
import itertools
from collections import Counter
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk import FreqDist
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

import plotly.express as px
import plotly.offline as py #visualization
import plotly.graph_objs as go
from plotly.offline import *

In [ ]:
#Clean Columns name
def Nombre_Limpio(text):
  words=text.split("_")
  s= " ".join(words)
  return s
# Detect Outliers at the data
def Outliers(df,cols):
  df_copy=df.copy()
  for col in cols:
    ValMax=df_copy[col].quantile(0.75)+1.5*(df_copy[col].quantile(0.75)-df_copy[col].quantile(0.25))
    df_copy=df_copy[df_copy[col]<ValMax].reset_index(drop=True)
  return df_copy
# Generate Graphs
def GraficosConteos(df,w):
  Prueba=df[w].value_counts().reset_index()
  Prueba.columns=[w,"Cantidad de registros"]
  Prueba=Prueba.sort_values(by="Cantidad de registros",ascending=False)
  fig=px.bar(Prueba[:20],x=w,y="Cantidad de registros",text="Cantidad de registros", title=f"Gráfico de conteo {w}")
  fig.write_image(f"images/Gráfico de conteo {w}.png")
  fig.show()
def GraficosBoxplot(df,w):
  fig=px.box(df,y=w, title=f"Gráfico de boxplot {w}")
  fig.write_image(f"images/Gráfico de boxplot {w}.png")
  fig.show()
# Frecuency words
def CommonWords(df):
  global df_dist
  letters_only = re.sub("[^a-zA-Z]", " ", df)
  words = letters_only.lower().split()
  stops = set(stopwords.words("spanish"))
  stops.update(['n','t','p','d','th'])
  meaningful_words = [w for w in words if not w in stops]
  fdist = FreqDist(meaningful_words)
  df_dist = pd.DataFrame.from_dict(fdist, orient='index')
  df_dist=df_dist.reset_index()
  df_dist.columns=["Palabra","Frecuencia"]
  df_dist=df_dist.sort_values(by="Frecuencia",ascending=False).reset_index(drop= True)
  fig=px.bar(df_dist[:20],x="Palabra",y="Frecuencia",text="Frecuencia", title="Gráfico de Frecuencia de palabras frecuentes")
  fig.write_image(f"images/Gráfico de Frecuencia de palabras frecuentes.png")
  fig.show()
  return df_dist
def Frequency(Negative_list):
  Negative_After_df=pd.DataFrame.from_dict(Counter(Negative_list), orient='index').reset_index()
  Negative_After_df.columns=["Word","Frecuencia"]
  Negative_After_df=Negative_After_df.sort_values(by="Frecuencia", ascending= False).reset_index(drop=True)
  return Negative_After_df

#Find the Ngrams
def ngrams(lst, n):
    tlst = lst
    while True:
       a, b = tee(tlst)
       l = tuple(islice(a, n))
       if len(l) == n:
         yield l ## Crea un generador, metodo que contiene todo los ngrams que se pudieron sacar.  
         next(b)
         tlst = b
       else:
        break
# Show the Ngrams       
def Ndiagrams(df,number):
  """ It requieres a dataframe and the number of ngrams
      It returns a dataframe with the ngrams frequency
  df=DataFrame
  number= Number of ngrams required
  """
  letters_only = re.sub("[^a-zA-Z]", " ", df)
  words = letters_only.lower().split()
  stops = set(stopwords.words("spanish"))
  stops.update(['n','t','p','d','th'])
  test = [w for w in words if not w in stops]
  Ngrams=Counter(ngrams(test, number))
  final_df=pd.DataFrame.from_dict(Ngrams, orient='index').reset_index()
  final_df.columns=[f"{number}-gram","Frecuencia"]
  final_df=final_df.sort_values(by="Frecuencia", ascending= False).reset_index(drop=True)
  final_df[f"{number}-gram"]=final_df[f"{number}-gram"].apply(lambda x:str(x))
  fig=px.bar(final_df[:20],x=f"{number}-gram",y="Frecuencia",text="Frecuencia", title=f"Gráfico de Frecuencia de {number}-gram")
  fig.write_image(f"images/Gráfico de Frecuencia de {number}-gram.png")
  fig.show()
  
  return final_df


In [ ]:
client = Socrata("www.datos.gov.co", None)

results = client.get("xvdy-vvsk",limit=100000)

df_secopi = pd.DataFrame.from_records(results)
df_secopi.reset_index(drop=True)
df_secopi.columns

In [ ]:
ProfileReport(df_secopi, title="Pre analisis Secop I").to_file("PreAnalisis Secop I.html")

** Hay que cambiar la naturaleza de las variables. Todas salieron objeto debido a que son sacadas de un API

1. Hay datos desde el 2011 o 2010
2. Hay una gran cantidad de contratos que no tienen un año confirmado del contrato lo tienen catalogado como 0.
3. Existe una gran concentración (80%) de los contratos en territorial y un 19% en nacional. Sugiero quitar los que no estan definidos (0.1%)
4. La entidad de territorial distrital municipal se divide en 6 niveles donde el lvl 6 es el que más tiene contratos(19%) seguido por el 5(2.8%) y el 4(2.3%).
5. No creo que el nit de la entidad o el nombre especifico de esta sirva como variable a analizar por la alta cardinalidad. Existen 2646 entidades unicas en 10k de datos, sin embargo hay menos nits que nombres de empresa. Esto quiere decir que hay empresas no registradas con NIT.
6. Existe una concentracion en los procesos 12, 4 y 13 de casi el 93.6%. Contratacion Directa, Regin Especial y Contratación de minima cuantia son los tipos de procesos que sobre salen.
7. Más de la mitad de los contratos (59.4%) son conratos celebrados. Mientras que el 26.6% de los contratos son liquidados. Solo el 7.7% de los contratos estan en convocatoria. El otro 6% Son contratos que no hay finalizado.
8. Prestación de servicios se divide practicamente en 2 los que no tienen ninguno o fueron presentados por profesionales. 
9. El regimen contractual es principalmente de Estatuto General de contratación , id=3. (71.8%) o de Regimen Especial id=2 (28%)
10. El objeto contractual que tienen mayor cantidad de registros es de Gestión Pública(41.7%) y Servicios de Salud(11.2%).





# 2.Importe de Archivos SECOP II




In [ ]:
!pip install sodapy
import pandas as pd
from sodapy import Socrata


client = Socrata("www.datos.gov.co", None)


results = client.get("jbjy-vk9h", limit=100000)

df_secopii = pd.DataFrame.from_records(results)

In [ ]:
ProfileReport(df_secopii, title="Pre analisis Secop II").to_file("PreAnalisis Secop II.html")

# 3.Tienda Virtual

In [ ]:
client = Socrata("www.datos.gov.co", None)


results = client.get("rgxm-mmea", limit=20000)

df_tienda = pd.DataFrame.from_records(results)

In [ ]:
clean_tienda=df_tienda.drop(columns=['espostconflicto','nit_proveedor'])
cols_nums3=["a_o","total"]
fechas_tienda=["fecha"]
for x in cols_nums3:
  clean_tienda[x]=clean_tienda[x].apply(lambda y: float(y))
for x in fechas_tienda:
  clean_tienda[x]=clean_tienda[x].apply(lambda y: datetime.strptime(str(y[:10]),"%Y-%m-%d"))

In [ ]:
ProfileReport(df_tienda, title="Pre analisis Tienda Virtual").to_file("PreAnalisis Tienda Virtual.html")

# Cleaning Data

## Secop I


In [ ]:
#Identify Cols
cols_nums1= ["cuantia_proceso","tiempo_adiciones_en_dias","cuantia_contrato","valor_total_de_adiciones","valor_contrato_con_adiciones","plazo_de_ejec_del_contrato"]
no_cols= ['anno_cargue_secop','nit_de_la_entidad',"id_tipo_de_proceso","id_regimen_de_contratacion","id_objeto_a_contratar","fecha_de_cargue_en_el_secop",
           'numero_de_constancia','id_grupo','id_familia','id_clase','id_ajudicacion','tipo_doc_representante_legal',
           'identific_del_represen_legal', 'nombre_del_represen_legal','rango_de_ejec_del_contrato','compromiso_presupuestal','id_origen_de_los_recursos',
           'proponentes_seleccionados','calificacion_definitiva','id_sub_unidad_ejecutora','nombre_sub_unidad_ejecutora',
           'espostconflicto','marcacion_adiciones','posicion_rubro', 'nombre_rubro', 'valor_rubro','sexo_replegal_entidad', 'pilar_acuerdo_paz', 
           'punto_acuerdo_paz','departamento_entidad', 'ultima_actualizacion','numero_de_proceso','numero_del_contrato',
          'tiempo_adiciones_en_meses','origen_de_los_recursos','codigo_bpin',	'ruta_proceso_en_secop_i','moneda','nivel_entidad','orden_entidad','objeto_del_contrato_a_la'	]
fechas_secop1= [ 'fecha_de_firma_del_contrato', 'fecha_ini_ejec_contrato','fecha_fin_ejec_contrato']

In [ ]:
#Identify Null values
df_secopi[fechas_secop1].isna().any()

In [ ]:
null_columns=df_secopi.columns[df_secopi.isnull().any()]
df_secopi[null_columns].isnull().sum()/len(df_secopi)

In [ ]:
#Impute values
clean_secop1=df_secopi.drop(columns=no_cols,axis=1)
clean_secop1=clean_secop1.dropna()


In [ ]:
# Cast variables to acording to their nature
for x in cols_nums1:
  clean_secop1[x]=clean_secop1[x].apply(lambda y: float(y))
for x in fechas_secop1:
  clean_secop1[x]=clean_secop1[x].apply(lambda y: datetime.strptime(str(y),"%m/%d/%Y"))

In [ ]:
#Lower main variable
clean_secop1.detalle_del_objeto_a_contratar=clean_secop1.detalle_del_objeto_a_contratar.apply(lambda x: x.lower())


In [ ]:
# Cambio Formato de los titulos 
cols_dirty_secopi=clean_secop1.columns.to_list()
cols_clean_secopi=list (map(Nombre_Limpio,cols_dirty_secopi))
clean_secop1.columns=cols_clean_secopi
cols_nums1=list (map(Nombre_Limpio,cols_nums1))
col_dates1 = list (map(Nombre_Limpio,fechas_secop1))
col_dates1

In [ ]:
# Identify Outliers, days are too concentrated at the right that's why the filters is different for this variable.
cleani_sindias=Outliers(clean_secop1,["cuantia proceso","cuantia contrato",'valor contrato con adiciones','plazo de ejec del contrato'])
cleani=cleani_sindias[cleani_sindias['tiempo adiciones en dias']<cleani_sindias['tiempo adiciones en dias'].quantile(0.99)].reset_index(drop=True)
cleani

In [ ]:
#Select columns string variables to graph
colsi=clean_secop1.columns.to_list()
colsi=[x for x in colsi[1:] if x not in col_dates1 and x not in cols_nums1 and x not in ['c digo de la entidad','identificacion del contratista']]
colsi

In [ ]:
#Graph Frecuency Plots
for col in colsi:
  GraficosConteos(cleani,col)

In [ ]:
#Graph Boxplot
for col in cols_nums1:
  GraficosBoxplot(cleani,col)

##Cleaning Secop II

In [ ]:

cols_no_ii=['nit_entidad','orden','referencia_del_contrato','condiciones_de_entrega','tipodocproveedor','obligaci_n_ambiental','valor_de_pago_adelantado','valor_pendiente_de_pago',
            'valor_amortizado','c_digo_bpin', 'anno_bpin','urlproceso','puntos_del_acuerdo','pilares_del_acuerdo','proceso_de_compra','id_contrato','codigo_de_categoria_principal',
            'justificacion_modalidad_de','documento_proveedor','habilita_pago_adelantado'	,'liquidaci_n'	,'obligaciones_postconsumo','reversion','valor_pendiente_de','saldo_vigencia',
            'espostconflicto','estado_bpin']
cols_nums2=['valor_del_contrato','valor_del_contrato','valor_facturado', 'valor_pagado', 'valor_pendiente_de_ejecucion','saldo_cdp','dias_adicionados']
cols_date2=["fecha_de_firma" ,'fecha_de_inicio_del_contrato', 'fecha_de_fin_del_contrato','fecha_de_inicio_de_ejecucion', 'fecha_de_fin_de_ejecucion']



In [ ]:
clean_secop2=df_secopii.drop(columns=cols_no_ii)
clean_secop2=clean_secop2.dropna()

In [ ]:
# Cast variables to acording to their nature
for x in cols_nums2:
  clean_secop2[x]=clean_secop2[x].apply(lambda y: float(y))
for x in cols_date2:
  clean_secop2[x]=clean_secop2[x].apply(lambda y: datetime.strptime(str(y)[:10],"%Y-%m-%d"))
clean_secop2.descripcion_del_proceso	=clean_secop2.descripcion_del_proceso.apply(lambda x: x.lower())

In [ ]:
#Cleaning tittles of columns
cols_dirty_secopii=clean_secop2.columns.to_list()
cols_clean_secopii=list (map(Nombre_Limpio,cols_dirty_secopii))
clean_secop2.columns=cols_clean_secopii
cols_nums2=list (map(Nombre_Limpio,cols_nums2))
col_dates2 = list (map(Nombre_Limpio,cols_date2))
col_dates2

In [ ]:
# Identify Outliers, days are too concentrated at the right that's why the filters is different for this variable.
cleanii_sindias=Outliers(clean_secop2,['valor del contrato','valor facturado', 'valor pagado', 'valor pendiente de ejecucion','saldo cdp'])
cleanii=cleanii_sindias[cleanii_sindias['dias adicionados']<cleanii_sindias['dias adicionados'].quantile(0.99)].reset_index(drop=True)
cleanii


In [ ]:
#Select columns string variables to graph
colsii=clean_secop1.columns.to_list()
colsii=[x for x in cols_clean_secopii if x not in col_dates2 and x not in cols_nums2]
colsii

In [ ]:
for col in colsii:
  GraficosConteos(cleanii,col)

In [ ]:
for col in cols_nums2:
  GraficosBoxplot(cleanii,col)

#EDA Advanced

In [ ]:

#Correlation Secop I 
correlation = cleani.corr()
matrix_cols = correlation.columns.tolist()
corr_array  = np.array(correlation)

#Plotting
trace = go.Heatmap(z = corr_array,
                   x = matrix_cols,
                   y = matrix_cols,
                   colorscale = "Viridis",
                   colorbar   = dict(title = "Pearson Correlation coefficient",
                                     titleside = "right"
                                    ) ,
                  )

layout = go.Layout(dict(title = " Matrix Correlaccion Matrix for SecopI",
                        autosize = True,
                        height  = 720,
                        width   = 800,
                        margin  = dict(r = 0 ,l = 210,
                                       t = 25,b = 210,
                                      ),
                        yaxis   = dict(tickfont = dict(size = 9)),
                        xaxis   = dict(tickfont = dict(size = 9))
                       )
                  )

data = [trace]
fig = go.Figure(data=data,layout=layout)
fig.write_image(f"images/matriz de secopI.png")
py.iplot(fig)

In [ ]:
#Correlation Secop II 
correlation = cleanii.corr()
matrix_cols = correlation.columns.tolist()
corr_array  = np.array(correlation)

#Plotting
trace = go.Heatmap(z = corr_array,
                   x = matrix_cols,
                   y = matrix_cols,
                   colorscale = "Viridis",
                   colorbar   = dict(title = "Pearson Correlation coefficient",
                                     titleside = "right"
                                    ) ,
                  )

layout = go.Layout(dict(title = " Matrix Correlaccion Matrix for SecopII",
                        autosize = True,
                        height  = 720,
                        width   = 800,
                        margin  = dict(r = 0 ,l = 210,
                                       t = 25,b = 210,
                                      ),
                        yaxis   = dict(tickfont = dict(size = 9)),
                        xaxis   = dict(tickfont = dict(size = 9))
                       )
                  )

data = [trace]
fig = go.Figure(data=data,layout=layout)
fig.write_image(f"images/matriz de secopII.png")
py.iplot(fig)

In [ ]:
#Frecuency Tables Secop I and II
Descripciones_secopi=cleani["detalle del objeto a contratar"].to_string()
DetalleI=CommonWords(Descripciones_secopi)
DetalleI

In [ ]:
Descripciones_secopii=cleanii["descripcion del proceso"].to_string()
DetalleII=CommonWords(Descripciones_secopii)
DetalleII

In [ ]:
#N gramas Secop I and II
SecondSecopI= Ndiagrams(Descripciones_secopi,2)
ThirdSecopI= Ndiagrams(Descripciones_secopi,3)

In [ ]:
SecondSecopII= Ndiagrams(Descripciones_secopii,2)
ThirdSecopII= Ndiagrams(Descripciones_secopii,3)


Tienda Virtual

Graficas de conteo por variable